# STEP 4 Part 2 - Dynamic Windowing

This notebook contains the first tests that were executed when working with the dynamic windowing.
Many different models were trained.
The code is deprecated and uncommented but can be used to see how the different models perform in comparison.

## Imports

"timeseries_dataset_from_array" is a function that is not part of the used tf version but is required to transform the data, thus it is manually copied here and modified slightly

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import import_ipynb

In [2]:
from model_helper import ModelHelper

importing Jupyter notebook from model_helper.ipynb


The usual reading of the csv and setting of the vocabulary size

In [3]:
df = pd.read_csv("./4square/processed_transformed_locations.csv")
df.head(10)

,location_id,user_id,clock_sin,clock_cos,day_sin,day_cos,month_sin,month_cos,week_day_sin,week_day_cos
0,0,470,-1.000000,0.000654,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349
1,1,979,-0.999998,0.001818,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349
2,2,69,-0.999945,0.010472,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349
3,3,395,-0.999931,0.011708,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349
4,0,87,-0.999914,0.013090,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349
5,4,484,-0.999848,0.017452,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349
6,3,642,-0.999796,0.020215,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349
7,5,292,-0.999790,0.020506,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349
8,1,428,-0.999622,0.027485,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349
9,1,877,-0.999620,0.027558,0.587785,0.809017,0.866025,-0.5,0.781831,0.62349


In [4]:
locations = df.location_id
vocab_size = locations.nunique()
vocab_size

141

In [9]:
mh = ModelHelper(df, 17)
mh.set_client_column_name('user_id')

Generate location sequences.

In [11]:
mh.set_target_column_name('location_id')
mh.set_vocab_size()
mh.vocab_size

141

In [13]:
mh.split_concat_user_df()
mh.df_train

AttributeError: 'ModelHelper' object has no attribute 'users_locations'

The WindowGenerator class gets the train/val/test data as well as several other parameters:
* input_width - defines the length of the input as part of the window
* label_width - defines the length of the label (prediction target) as part of the window, can be used to predict multiple time steps in the future
* shift - offsets the label by the respective number of (time) steps
* label_columns - the columns used for the prediction

Examples found below.

In [7]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=train_df, val_df=val_df, test_df=test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

Exemplary window, output shows the indices.

In [8]:
w = WindowGenerator(input_width=16, label_width=1, shift=1,
                     label_columns=['cat_id'])
w

Total window size: 17
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
Label indices: [16]
Label column name(s): ['cat_id']

The 'split_window' function does the slicing of the dataset according to the window indices.

In [9]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

An example follows:

In [10]:
# Stack three slices, the length of the total window.
example_window = tf.stack([np.array(mh.df_train[:w.total_window_size]),
                           np.array(mh.df_train[100:100+w.total_window_size]),
                           np.array(mh.df_train[200:200+w.total_window_size])])

example_inputs, example_labels = w.split_window(example_window)

print('All shapes are: (batch, time, features)')
print(f'Window shape: {example_window.shape}')
print(f'Inputs shape: {example_inputs.shape}')
print(f'Labels shape: {example_labels.shape}')

All shapes are: (batch, time, features)
Window shape: (3, 17, 9)
Inputs shape: (3, 16, 9)
Labels shape: (3, 1, 1)


The 'make_dataset' function executes the self-imported 'timeseries_dataset_from_array' function, then splits the data in the respective windows.
The "batch_size" can be defined here. Also, the "sequence_stride" is an important parameter.
It can be used to leave out windows and thus reduce the number of generated sequences.
This can be useful if lots of data is available.
With a sequence_stride of 1 the window is moved to the next starting index, With a sequence stride of 2 the next is skipped.
As a result, only half of the sequences are created.

In [11]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = mh.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=8,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

Assign properties to create the dataset.

In [12]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [13]:
# Each element is an (inputs, label) pair.
w.train.element_spec

(TensorSpec(shape=(None, 16, 9), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 1, 1), dtype=tf.float32, name=None))

In [14]:
column_indices = {name: i for i, name in enumerate(df.columns)}

Compile and fit the model with the usual metrics and loss function.

In [15]:
MAX_EPOCHS = 20

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')
  early_stopping2 = tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy',
                                                    patience=patience,
                                                    mode='max')

  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.002),)

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      #callbacks=[early_stopping2]
                      )
  return history

In [16]:
single_step_window = WindowGenerator(
    input_width=1, label_width=1, shift=1,
    label_columns=['cat_id'])
single_step_window

Total window size: 2
Input indices: [0]
Label indices: [1]
Label column name(s): ['cat_id']

It follows some testing with simple networks such as:
* ordinary Dense Networks
* Convolutional Networks
* LSTMs
* GRUs

In [17]:
dense = tf.keras.Sequential([
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=vocab_size)
])

history = compile_and_fit(dense, single_step_window)

val_performance = dense.evaluate(single_step_window.val)
performance = dense.evaluate(single_step_window.test, verbose=0)

Epoch 1/20
236/236 [==============================] - 0s 1ms/step - loss: 2.2965 - sparse_categorical_accuracy: 0.2566 - val_loss: 2.1963 - val_sparse_categorical_accuracy: 0.2579
Epoch 2/20
236/236 [==============================] - 0s 932us/step - loss: 1.9902 - sparse_categorical_accuracy: 0.3155 - val_loss: 2.1776 - val_sparse_categorical_accuracy: 0.2375
Epoch 3/20
236/236 [==============================] - 0s 750us/step - loss: 1.8852 - sparse_categorical_accuracy: 0.3494 - val_loss: 2.1756 - val_sparse_categorical_accuracy: 0.2486
Epoch 4/20
236/236 [==============================] - 0s 809us/step - loss: 1.8247 - sparse_categorical_accuracy: 0.3484 - val_loss: 2.1757 - val_sparse_categorical_accuracy: 0.2523
Epoch 5/20
236/236 [==============================] - 0s 771us/step - loss: 1.7718 - sparse_categorical_accuracy: 0.3786 - val_loss: 2.2241 - val_sparse_categorical_accuracy: 0.2338
Epoch 6/20
236/236 [==============================] - 0s 758us/step - loss: 1.7186 - sparse_

In [18]:
my_window = WindowGenerator(
    input_width=16, label_width=1, shift=1,
    label_columns=['cat_id'])

my_window

Total window size: 17
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
Label indices: [16]
Label column name(s): ['cat_id']

In [19]:
dense2 = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=vocab_size)
])

history = compile_and_fit(dense2, my_window)

val_performance = dense2.evaluate(my_window.val)
performance = dense2.evaluate(my_window.test, verbose=0)

Epoch 1/20
234/234 [==============================] - 0s 915us/step - loss: 2.3024 - sparse_categorical_accuracy: 0.2742 - val_loss: 2.3626 - val_sparse_categorical_accuracy: 0.2156
Epoch 2/20
234/234 [==============================] - 0s 778us/step - loss: 1.9594 - sparse_categorical_accuracy: 0.3378 - val_loss: 2.4278 - val_sparse_categorical_accuracy: 0.1908
Epoch 3/20
234/234 [==============================] - 0s 709us/step - loss: 1.7845 - sparse_categorical_accuracy: 0.3891 - val_loss: 2.3818 - val_sparse_categorical_accuracy: 0.2118
Epoch 4/20
234/234 [==============================] - 0s 718us/step - loss: 1.6248 - sparse_categorical_accuracy: 0.4201 - val_loss: 2.5073 - val_sparse_categorical_accuracy: 0.1870
Epoch 5/20
234/234 [==============================] - 0s 983us/step - loss: 1.4506 - sparse_categorical_accuracy: 0.4837 - val_loss: 2.6160 - val_sparse_categorical_accuracy: 0.1927
Epoch 6/20
234/234 [==============================] - 0s 782us/step - loss: 1.2735 - spars

In [20]:
another_window = WindowGenerator(
    input_width=32, label_width=1, shift=1,
    label_columns=['cat_id'])

another_window

Total window size: 33
Input indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31]
Label indices: [32]
Label column name(s): ['cat_id']

In [21]:
dense3 = tf.keras.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=vocab_size)
])

history = compile_and_fit(dense3, another_window)

val_performance = dense3.evaluate(another_window.val)
performance = dense3.evaluate(another_window.test, verbose=0)

Epoch 1/20
232/232 [==============================] - 0s 884us/step - loss: 2.4233 - sparse_categorical_accuracy: 0.2469 - val_loss: 2.3766 - val_sparse_categorical_accuracy: 0.2087
Epoch 2/20
232/232 [==============================] - 0s 759us/step - loss: 2.0182 - sparse_categorical_accuracy: 0.3191 - val_loss: 2.4397 - val_sparse_categorical_accuracy: 0.1890
Epoch 3/20
232/232 [==============================] - 0s 754us/step - loss: 1.8660 - sparse_categorical_accuracy: 0.3509 - val_loss: 2.5956 - val_sparse_categorical_accuracy: 0.2028
Epoch 4/20
232/232 [==============================] - 0s 746us/step - loss: 1.6995 - sparse_categorical_accuracy: 0.4205 - val_loss: 2.5933 - val_sparse_categorical_accuracy: 0.1752
Epoch 5/20
232/232 [==============================] - 0s 716us/step - loss: 1.5141 - sparse_categorical_accuracy: 0.4836 - val_loss: 2.6886 - val_sparse_categorical_accuracy: 0.1673
Epoch 6/20
232/232 [==============================] - 0s 759us/step - loss: 1.3511 - spars

In [22]:
CONV_WIDTH = 10
conv_window = WindowGenerator(
    input_width=CONV_WIDTH,
    label_width=1,
    shift=1,
    label_columns=['cat_id'])

In [23]:
conv_model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32,
                           kernel_size=(CONV_WIDTH,),
                           activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=vocab_size)
])


history = compile_and_fit(conv_model, conv_window)

val_performance = conv_model.evaluate(conv_window.val)
performance = conv_model.evaluate(conv_window.test, verbose=0)

Epoch 1/20
235/235 [==============================] - 0s 1ms/step - loss: 2.2307 - sparse_categorical_accuracy: 0.2616 - val_loss: 2.2304 - val_sparse_categorical_accuracy: 0.2358
Epoch 2/20
235/235 [==============================] - 0s 770us/step - loss: 1.9956 - sparse_categorical_accuracy: 0.3335 - val_loss: 2.1990 - val_sparse_categorical_accuracy: 0.2340
Epoch 3/20
235/235 [==============================] - 0s 783us/step - loss: 1.8728 - sparse_categorical_accuracy: 0.3596 - val_loss: 2.2241 - val_sparse_categorical_accuracy: 0.2226
Epoch 4/20
235/235 [==============================] - 0s 770us/step - loss: 1.7822 - sparse_categorical_accuracy: 0.3889 - val_loss: 2.2195 - val_sparse_categorical_accuracy: 0.2189
Epoch 5/20
235/235 [==============================] - 0s 826us/step - loss: 1.7092 - sparse_categorical_accuracy: 0.4012 - val_loss: 2.2309 - val_sparse_categorical_accuracy: 0.2208
Epoch 6/20
235/235 [==============================] - 0s 757us/step - loss: 1.6354 - sparse_

In [24]:
window_3 = WindowGenerator(
    input_width=8, label_width=1, shift=1,
    label_columns=['cat_id'])

window_3

Total window size: 9
Input indices: [0 1 2 3 4 5 6 7]
Label indices: [8]
Label column name(s): ['cat_id']

In [25]:
# Each element is an (inputs, label) pair.
window_3.train.element_spec

(TensorSpec(shape=(None, 8, 9), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 1, 1), dtype=tf.float32, name=None))

In [26]:
lstm = tf.keras.Sequential()
lstm.add(tf.keras.layers.LSTM(128,return_sequences=True,input_shape=(8, 9), activation='relu'))
lstm.add(tf.keras.layers.LSTM(64,input_shape=(8, 9), activation='relu'))
lstm.add(tf.keras.layers.Dropout(0.5))
lstm.add(tf.keras.layers.Dense(vocab_size))

history = compile_and_fit(lstm, window_3)

val_performance = lstm.evaluate(window_3.val)
performance = lstm.evaluate(window_3.test, verbose=0)

Epoch 1/20
235/235 [==============================] - 1s 5ms/step - loss: 2.4130 - sparse_categorical_accuracy: 0.2437 - val_loss: 2.2554 - val_sparse_categorical_accuracy: 0.2462
Epoch 2/20
235/235 [==============================] - 1s 5ms/step - loss: 2.1828 - sparse_categorical_accuracy: 0.2885 - val_loss: 2.2440 - val_sparse_categorical_accuracy: 0.2462
Epoch 3/20
235/235 [==============================] - 1s 5ms/step - loss: 2.1019 - sparse_categorical_accuracy: 0.3007 - val_loss: 2.2223 - val_sparse_categorical_accuracy: 0.2462
Epoch 4/20
235/235 [==============================] - 1s 4ms/step - loss: 2.0531 - sparse_categorical_accuracy: 0.3113 - val_loss: 2.1922 - val_sparse_categorical_accuracy: 0.2481
Epoch 5/20
235/235 [==============================] - 1s 5ms/step - loss: 2.0140 - sparse_categorical_accuracy: 0.3124 - val_loss: 2.1794 - val_sparse_categorical_accuracy: 0.2256
Epoch 6/20
235/235 [==============================] - 1s 4ms/step - loss: 1.9880 - sparse_categorica

In [27]:
gru1 = tf.keras.Sequential()
gru1.add(tf.keras.layers.GRU(128,return_sequences=True,input_shape=(8, 9), activation='relu'))
gru1.add(tf.keras.layers.GRU(64,input_shape=(8, 9), activation='relu'))
gru1.add(tf.keras.layers.Dropout(0.5))
gru1.add(tf.keras.layers.Dense(vocab_size))

history = compile_and_fit(gru1, window_3)

val_performance = gru1.evaluate(window_3.val)
performance = gru1.evaluate(window_3.test, verbose=0)

Epoch 1/20
235/235 [==============================] - 1s 4ms/step - loss: 2.4007 - sparse_categorical_accuracy: 0.2576 - val_loss: 2.2953 - val_sparse_categorical_accuracy: 0.2444
Epoch 2/20
235/235 [==============================] - 1s 4ms/step - loss: 2.1227 - sparse_categorical_accuracy: 0.2906 - val_loss: 2.1800 - val_sparse_categorical_accuracy: 0.2425
Epoch 3/20
235/235 [==============================] - 1s 3ms/step - loss: 2.0210 - sparse_categorical_accuracy: 0.3321 - val_loss: 2.1677 - val_sparse_categorical_accuracy: 0.2556
Epoch 4/20
235/235 [==============================] - 1s 3ms/step - loss: 1.9482 - sparse_categorical_accuracy: 0.3363 - val_loss: 2.1346 - val_sparse_categorical_accuracy: 0.2594
Epoch 5/20
235/235 [==============================] - 1s 4ms/step - loss: 1.8953 - sparse_categorical_accuracy: 0.3513 - val_loss: 2.1367 - val_sparse_categorical_accuracy: 0.2669
Epoch 6/20
235/235 [==============================] - 1s 4ms/step - loss: 1.8426 - sparse_categorica

In [28]:
gru2 = tf.keras.Sequential()
gru2.add(tf.keras.layers.GRU(128,return_sequences=True,input_shape=(1, 9), activation='relu'))
gru2.add(tf.keras.layers.GRU(64,input_shape=(1, 9), activation='relu'))
gru2.add(tf.keras.layers.Dropout(0.5))
gru2.add(tf.keras.layers.Dense(vocab_size))

history = compile_and_fit(gru2, single_step_window)

val_performance = gru2.evaluate(single_step_window.val)
performance = gru2.evaluate(single_step_window.test, verbose=0)

Epoch 1/20
236/236 [==============================] - 1s 2ms/step - loss: 2.4968 - sparse_categorical_accuracy: 0.2333 - val_loss: 2.2547 - val_sparse_categorical_accuracy: 0.2301
Epoch 2/20
236/236 [==============================] - 0s 1ms/step - loss: 2.1330 - sparse_categorical_accuracy: 0.2847 - val_loss: 2.1773 - val_sparse_categorical_accuracy: 0.2430
Epoch 3/20
236/236 [==============================] - 0s 1ms/step - loss: 2.0627 - sparse_categorical_accuracy: 0.2959 - val_loss: 2.1671 - val_sparse_categorical_accuracy: 0.2356
Epoch 4/20
236/236 [==============================] - 0s 1ms/step - loss: 2.0167 - sparse_categorical_accuracy: 0.2948 - val_loss: 2.1752 - val_sparse_categorical_accuracy: 0.2393
Epoch 5/20
236/236 [==============================] - 0s 1ms/step - loss: 1.9738 - sparse_categorical_accuracy: 0.3277 - val_loss: 2.1985 - val_sparse_categorical_accuracy: 0.2486
Epoch 6/20
236/236 [==============================] - 0s 1ms/step - loss: 1.9285 - sparse_categorica